In [1]:
# coding: utf-8
import numpy as np
def vertices2DFl(beamThickness, hBeam, wChannel, hChannel, lInlet, lOutlet, nBeams, xSpacing):
    
    points = np.zeros((nBeams*18+6,3))
    
    #inlet
    points[1] = [0,0,-wChannel] 
    points[2] = [0,hBeam,0]
    points[3] = [0,hBeam,-wChannel] 
    points[4] = [0,hChannel,0]
    points[5] = [0,hChannel,-wChannel] 
    
    
    for j in range(nBeams):
        
        for k in range(18):
            
            i = (j * 18) + k + 6
                
            if (i%2 != 0):

                points[i][2] = -wChannel

            if ((i%6 == 2) or (i%6 == 3)):

                points[i][1] = hBeam

            if ((i%6 == 4) or (i%6 == 5)):

                points[i][1] = hChannel
                
            if (k < 6):
                
                points[i][0] = lInlet + j * (xSpacing + beamThickness) 
                    
            elif (k < 12):
                
                points[i][0] = lInlet + beamThickness + j * (xSpacing + beamThickness)

            else:
                
                points[i][0] = lInlet + beamThickness + xSpacing/2 + j * (xSpacing + beamThickness)
    
    points[-6] = [lInlet+lOutlet+(nBeams-1)*(beamThickness+xSpacing)+beamThickness,0,0] 
    points[-5] = [lInlet+lOutlet+(nBeams-1)*(beamThickness+xSpacing)+beamThickness,0,-wChannel] 
    points[-4] = [lInlet+lOutlet+(nBeams-1)*(beamThickness+xSpacing)+beamThickness,hBeam,0]
    points[-3] = [lInlet+lOutlet+(nBeams-1)*(beamThickness+xSpacing)+beamThickness,hBeam,-wChannel] 
    points[-2] = [lInlet+lOutlet+(nBeams-1)*(beamThickness+xSpacing)+beamThickness,hChannel,0]
    points[-1] = [lInlet+lOutlet+(nBeams-1)*(beamThickness+xSpacing)+beamThickness,hChannel,-wChannel] 
    
    return points
    
def vertices2DS(beamThickness, hBeam, wChannel, lInlet, lOutlet, nBeams, xSpacing):
    
    points = np.array(np.zeros((nBeams*8,3)))
    #inlet
    points[0] = [lInlet,0,0] 
    points[1] = [lInlet,0,-wChannel] 
    points[2] = [lInlet,hBeam,0]
    points[3] = [lInlet,hBeam,-wChannel] 
    points[4] = [lInlet+beamThickness,0,0]
    points[5] = [lInlet+beamThickness,0,-wChannel] 
    points[6] = [lInlet+beamThickness,hBeam,0] 
    points[7] = [lInlet+beamThickness,hBeam,-wChannel]
    
    for i in np.arange(8,nBeams*8):
        
        points[i] = points[i%8]+[(xSpacing+beamThickness)*int(i/8),0,0]
        
    return points
    
def blocks2DFl(points, xProgression, beamYProgression, inletProgression, outletProgression, beamXRefinementFactor,mToNumCoeff):
    
    # (x Or y)Progression = measure of the degree in which the elements get refined as they
    # get closer to the beam region
    
    nBeams = int(points.shape[0]/18)
    blocks = np.array(np.zeros((int(nBeams*5),14)))
    
    blocks[0][:8] = [1 , 7 , 9 , 3 ,0 , 6 , 8 , 2 ] 
    blocks[1][:8] = [3 , 9 , 11, 5 ,2 , 8 , 10, 4 ] 
    blocks[2][:8] = [9 , 15, 17, 11,8 , 14, 16, 10 ]
    blocks[3][:8] = [13, 19, 21, 15,12, 18, 20, 14 ]
    blocks[4][:8] = [15, 21, 23, 17,14, 20, 22, 16 ]
    
    for i in np.arange(5,int(nBeams*5)):
        
        blocks[i][:8] = blocks[i%5][:8]+int(i/5)*18
    
    inletL = points[6][0]
    outletL = points[-1][0] - points[-7][0]
    hBeam = points[2][1]
    hChannel = points[4][1]
    xSpacing = points[18][0] - points[12][0]
    beamThickness = points[12][0] - points[6][0]
    inletNumElem = int(inletL * mToNumCoeff)
    outletNumElem = int(outletL * mToNumCoeff)
    beamXNumElem = int(beamXRefinementFactor * mToNumCoeff*beamThickness)
    beamYNumElem = int(hBeam * mToNumCoeff)
    beam2ChannelYNumElem = int((hChannel-hBeam) * mToNumCoeff)*0.75
    xSpacingNumElem = int(xSpacing * mToNumCoeff)
    
    blocks[0][8:] = [xSpacingNumElem , beamYNumElem , 1 , xProgression , beamYProgression , 1]
    blocks[1][8:] = [xSpacingNumElem , beam2ChannelYNumElem , 1 , xProgression ,(0.75/(beamYProgression)) , 1]
    blocks[2][8:] = [beamXNumElem , beam2ChannelYNumElem , 1 , 1 , (0.75/(beamYProgression))  , 1]
    blocks[3][8:] = [xSpacingNumElem , beamYNumElem , 1 , 1.0/xProgression, beamYProgression, 1]
    blocks[4][8:] = [xSpacingNumElem , beam2ChannelYNumElem , 1 ,1.0/xProgression, (0.75/(beamYProgression)), 1]
    
    for i in np.arange(5,int(nBeams*5)):
        
        blocks[i][8:] = blocks[i%5][8:]
    
    blocks[0][8:] = [int(inletNumElem/4) , beamYNumElem , 1 , inletProgression , beamYProgression , 1]
    blocks[1][8:] = [int(inletNumElem/4) , beam2ChannelYNumElem , 1 , inletProgression , (0.75/(beamYProgression)) , 1]
    blocks[-2][8:] = [int(outletNumElem/8) , beamYNumElem , 1 ,1.0/outletProgression , beamYProgression , 1]
    blocks[-1][8:] = [int(outletNumElem/8) , beam2ChannelYNumElem , 1 ,1.0/outletProgression,(0.75/(beamYProgression)) , 1]
    
    return blocks

def blocks2DS(points, beamYProgression, beamXRefinementFactor,mToNumCoeff):
    
    # (x Or y)Progression = measure of the degree in which the elements get refined as they
    # get closer to the beam region
    
    nBeams = np.int(points.shape[0]/8)
    blocks = np.array(np.zeros((int(nBeams),14)))
    
    blocks[0][:8] = [1 , 5 , 7 , 3,  0 , 4 , 6 , 2]
    
    for i in np.arange(1,int(nBeams)):
        
        blocks[i][:8] = blocks[i-1][:8]+8
    
    hBeam = points[2][1]
    beamThickness = points[4][0] - points[0][0]
    beamXNumElem = int(beamXRefinementFactor * mToNumCoeff*beamThickness)
    beamYNumElem = int(hBeam * mToNumCoeff)
    
    for i in range(nBeams):
        
        blocks[i][8:] = [beamXNumElem , beamYNumElem , 1 , 1 , beamYProgression , 1]
    
    return blocks
        
def facesFl(points):
    
    nBeams = np.int(points.shape[0]/18)
    interfacePerBeam = 3
    bottomPerBeam = 2
    topPerBeam = 3
    frontPerBeam = 5
    backPerBeam = 5
    interface = np.array(np.zeros((nBeams*interfacePerBeam,4)))
    bottom = np.array(np.zeros((nBeams*bottomPerBeam,4)))
    top = np.array(np.zeros((nBeams*topPerBeam,4)))
    inlet = np.array(np.zeros((2,4)))
    outlet = np.array(np.zeros((2,4)))
    front = np.array(np.zeros((nBeams*frontPerBeam,4)))
    back = np.array(np.zeros((nBeams*backPerBeam,4)))
    
    inlet[0] = [0,1,3,2]
    inlet[1] = [2,3,5,4]
    outlet[0] = inlet[0]+18*nBeams
    outlet[1] = inlet[1]+18*nBeams
    
    # init Interface
    interface[0] = [7,6,8,9]
    interface[1] = [8,14,15,9]
    interface[2] = [12,13,15,14]
    
    # init Bottom
    bottom[0] = [0,6,7,1]
    bottom[1] = [12,18,19,13]
    
    # init Top
    top[0] = [4,5,11,10]
    top[1] = [10,11,17,16]
    top[2] = [16,17,23,22]
    
    # init Interface
    front[0] = [0,2,8,6]
    front[1] = [2,4,10,8]
    front[2] = [8,10,16,14]
    front[3] = [12,14,20,18]
    front[4] = [14,16,22,20]
    
    # init Interface
    back[0] = [7,9,3,1]
    back[1] = [9,11,5,3]
    back[2] = [15,17,11,9]
    back[3] = [19,21,15,13]
    back[4] = [21,23,17,15]
    
    for i in np.arange(1,nBeams):

        for j in range(interfacePerBeam):

            interface[i*interfacePerBeam+j] = interface[j]+18*i
    
    
        for j in range(bottomPerBeam):

            bottom[i*bottomPerBeam+j] = bottom[j]+18*i
    
    
        for j in range(topPerBeam):

            top[i*topPerBeam+j] = top[j]+18*i
    
    
        for j in range(frontPerBeam):

            front[i*frontPerBeam+j] = front[j]+18*i
    
    
        for j in range(backPerBeam):

            back[i*backPerBeam+j] = back[j]+18*i
    
    return interface, bottom, top, inlet, outlet, front, back

def facesS(points):
    
    nBeams = np.int(points.shape[0]/8)
    interfacePerBeam = 3
    bottomPerBeam = 1
    frontPerBeam = 1
    backPerBeam = 1
    interface = np.array(np.zeros((nBeams*interfacePerBeam,4)))
    bottom = np.array(np.zeros((nBeams*bottomPerBeam,4)))
    front = np.array(np.zeros((nBeams*frontPerBeam,4)))
    back = np.array(np.zeros((nBeams*backPerBeam,4)))
    
    # init Interface
    interface[0] = [0,1,3,2]
    interface[1] = [2,3,7,6]
    interface[2] = [6,7,5,4]
    
    # init Bottom
    bottom[0] = [0,4,5,1]
    
    # init front
    front[0] = [0,2,6,4]
    
    # init back
    back[0] = [5,7,3,1]
    
    for i in np.arange(1,nBeams):

        for j in range(interfacePerBeam):

            interface[i*interfacePerBeam+j] = interface[j]+8*i
    
    
        for j in range(bottomPerBeam):

            bottom[i*bottomPerBeam+j] = bottom[j]+8*i
    
    
        for j in range(frontPerBeam):

            front[i*frontPerBeam+j] = front[j]+8*i
    
    
        for j in range(backPerBeam):

            back[i*backPerBeam+j] = back[j]+8*i
    
    return interface, bottom, front, back


def writeBlockMeshFl(points,blocks):

    face = facesFl(points)
    
    blockMeshFileFl = "blockMeshDictF"
    
    f = open(blockMeshFileFl,"w+")  
    f.write("FoamFile\n{\nversion     2.0;\nformat      ascii;\nclass       dictionary;\nobject      blockMeshDict;\n}\n")
    f.write("convertToMeters 1;\n\n")
    f.write("vertices\n(\n")
    
    for i in range(points.shape[0]):
        
        f.write("({0} {1} {2})\n".format(np.around(points[i][0],3),np.around(points[i][1],3),np.around(points[i][2],3)))
    
    f.write(");\n")
    f.write("blocks\n(\n\n")
    
    for i in range(blocks.shape[0]):
        
        f.write(" hex ({0} {1} {2} {3} {4} {5} {6} {7}) ({8} {9} {10}) simpleGrading ({11:5.2f} {12:5.2f} {13:5.2f})\n".format(np.int(blocks[i][0]),
                                                                                                               np.int(blocks[i][1]),
                                                                                                               np.int(blocks[i][2]),
                                                                                                               np.int(blocks[i][3]),
                                                                                                               np.int(blocks[i][4]),
                                                                                                               np.int(blocks[i][5]),
                                                                                                               np.int(blocks[i][6]),
                                                                                                               np.int(blocks[i][7]),
                                                                                                               np.int(blocks[i][8]),
                                                                                                               np.int(blocks[i][9]),
                                                                                                               np.int(blocks[i][10]),
                                                                                                               blocks[i][11],
                                                                                                               blocks[i][12],
                                                                                                               blocks[i][13]))
    
    f.write(");\n")
    f.write("edges\n(\n);\n")
    f.write("boundary\n(\n")
    f.write("interface\n{\n")
    f.write("type wall;\n faces\n(\n")
    
    for i in range(face[0].shape[0]):
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[0][i][0]),
                                             np.int(face[0][i][1]),
                                             np.int(face[0][i][2]),
                                             np.int(face[0][i][3])))
    f.write(");\n}\n")
                
    f.write("bottom\n{\n")
    f.write("type wall;\n faces\n(\n")
    
    for i in range(face[1].shape[0]):
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[1][i][0]),
                                             np.int(face[1][i][1]),
                                             np.int(face[1][i][2]),
                                             np.int(face[1][i][3])))
    
    f.write(");\n}\n")
                
    f.write("top\n{\n")
    f.write("type wall;\n faces\n(\n")
    
    for i in range(face[2].shape[0]):
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[2][i][0]),
                                             np.int(face[2][i][1]),
                                             np.int(face[2][i][2]),
                                             np.int(face[2][i][3])))
    
    f.write(");\n}\n")
                
    f.write("inlet\n{\n")
    f.write("type patch;\n faces\n(\n")
    
    for i in range(face[3].shape[0]):
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[3][i][0]),
                                             np.int(face[3][i][1]),
                                             np.int(face[3][i][2]),
                                             np.int(face[3][i][3])))
    
    
    f.write(");\n}")
    
    f.write("outlet\n{\n")
    f.write("type patch;\n faces\n(\n")
    
    for i in range(face[4].shape[0]):
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[4][i][0]),
                                             np.int(face[4][i][1]),
                                             np.int(face[4][i][2]),
                                             np.int(face[4][i][3])))
    
    
    f.write(");\n}\n")
    
    f.write("front\n{\n")
    f.write("type empty;\n faces\n(\n")
    
    for i in range(face[5].shape[0]):
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[5][i][0]),
                                             np.int(face[5][i][1]),
                                             np.int(face[5][i][2]),
                                             np.int(face[5][i][3])))
    
    
    f.write(");\n}\n")
    
    f.write("back\n{\n")
    f.write("type empty;\n faces\n(\n")
    
    for i in range(face[6].shape[0]):
        
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[6][i][0]),
                                             np.int(face[6][i][1]),
                                             np.int(face[6][i][2]),
                                             np.int(face[6][i][3])))
    
    
    f.write(");\n}\n);\n")
    f.write("mergePatchPairs\n(\n);")
    
    f.close()
    pass

def writeBlockMeshS(points,blocks):

    face = facesS(points)
    
    blockMeshFileS = "blockMeshDictS"
    
    f = open(blockMeshFileS,"w+")  
    f.write("FoamFile\n{\nversion     2.0;\nformat      ascii;\nclass       dictionary;\nobject      blockMeshDict;\n}\n")
    f.write("convertToMeters 1;\n\n")
    f.write("vertices\n(\n")
    
    for i in range(points.shape[0]):
        
        f.write("({0} {1} {2})\n".format(np.around(points[i][0],2),np.around(points[i][1],2),np.around(points[i][2],2)))
    
    f.write(");\n")
    f.write("blocks\n(\n\n")
    
    for i in range(blocks.shape[0]):
        
        f.write(" hex ({0} {1} {2} {3} {4} {5} {6} {7}) ({8} {9} {10}) simpleGrading ({11:5.2f} {12:5.2f} {13:5.2f})\n".format(np.int(blocks[i][0]),
                                                                                                               np.int(blocks[i][1]),
                                                                                                               np.int(blocks[i][2]),
                                                                                                               np.int(blocks[i][3]),
                                                                                                               np.int(blocks[i][4]),
                                                                                                               np.int(blocks[i][5]),
                                                                                                               np.int(blocks[i][6]),
                                                                                                               np.int(blocks[i][7]),
                                                                                                               np.int(blocks[i][8]),
                                                                                                               np.int(blocks[i][9]),
                                                                                                               np.int(blocks[i][10]),
                                                                                                               blocks[i][11],
                                                                                                               blocks[i][12],
                                                                                                               blocks[i][13]))
    f.write(");\n")
    f.write("edges\n(\n);\n")
    f.write("boundary\n(\n")
    f.write("interface\n{\n")
    f.write("type wall;\n faces\n(\n")
    
    for i in range(face[0].shape[0]):
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[0][i][0]),
                                             np.int(face[0][i][1]),
                                             np.int(face[0][i][2]),
                                             np.int(face[0][i][3])))
    f.write(");\n}\n")
                
    f.write("bottom\n{\n")
    f.write("type wall;\n faces\n(\n")
    
    for i in range(face[1].shape[0]):
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[1][i][0]),
                                             np.int(face[1][i][1]),
                                             np.int(face[1][i][2]),
                                             np.int(face[1][i][3])))
    
    f.write(");\n}\n")
                
    f.write("front\n{\n")
    f.write("type empty;\n faces\n(\n")
    
    for i in range(face[2].shape[0]):
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[2][i][0]),
                                             np.int(face[2][i][1]),
                                             np.int(face[2][i][2]),
                                             np.int(face[2][i][3])))
    
    
    f.write(");\n}\n")
    
    f.write("back\n{\n")

    f.write("type empty;\n faces\n(\n")
    
    for i in range(face[3].shape[0]):
        
        
        f.write("({0} {1} {2} {3})\n".format(np.int(face[3][i][0]),
                                             np.int(face[3][i][1]),
                                             np.int(face[3][i][2]),
                                             np.int(face[3][i][3])))
    
    
    f.write(");\n}\n);\n")
    f.write("mergePatchPairs\n(\n);")
    
    f.close()
    pass
    
def writeUInletAndPoints(topVelocity,numPoints,hChannel,wChannel):

    U = np.zeros((numPoints,3))
    points = np.zeros((numPoints,3))
    U[:,0] = np.linspace(0,topVelocity,numPoints+1)[1:]
    points[:,1] = np.linspace(0,hChannel,numPoints+1)[1:]

    info_before_U = "FoamFile\n\
{\n\
    version     2.0;\n\
    format      ascii;\n\
    class       vectorAverageField;\n\
    object      values;\n\
}\n\
\n\
// Average\n\
(0 0 0)\n\
\n\
// Data on points\n\
%d \n\
(\n" % (numPoints*2)

    info_before_points = "FoamFile\n\
{\n\
    version     2.0;\n\
    format      ascii;\n\
    class       vectorField;\n\
    object      points;\n\
}\n\
\n\
(\n"  

    f_U = open('U',"w")
    f_U.write(info_before_U)
    for i in range(numPoints):
        f_U.write(('(%4.2f %4.2f %4.2f)\n') % (U[i][0],U[i][1],U[i][2] ))
    f_U.write('\n')
    for i in range(numPoints):
        f_U.write(('(%4.2f %4.2f %4.2f)\n') % (U[i][0],U[i][1],U[i][2] ))
    f_U.write('\n)\n')
    f_U.close()

    f_points = open('points',"w")
    f_points.write(info_before_points)
    for i in range(numPoints):
        f_points.write(('(%4.2f %4.2f %4.2f)\n') % (points[i][0],points[i][1],-wChannel))
    f_points.write('\n')
    for i in range(numPoints):
        f_points.write(('(%4.2f %4.2f %4.2f)\n') % (points[i][0],points[i][1],points[i][2] ))
    f_points.write('\n)\n')
    f_points.close()

    pass

In [8]:
# coding: utf-8
# Template script for FSI simulation of hairbeds 2D run
import os
homedir = os.environ['HOME']
dropbox_path = '%s/Dropbox/2DSims' % homedir
os.chdir(dropbox_path)
#**************************************************************
#********************** Model parameters **********************
#**************************************************************

# Beam spacing (m) 
# spacingX = [0.2,0.5,0.1]
spacingX = [0.2]

# Beam height (m)
# beamH = [0.2,0.35,0.5]
beamH = [0.2]

# Number of beams
# beamsN = [2,4,6,8]
beamsN = [2]

# Elastic modulus of the beams (Pa: kg/m/s^2)
# Emoduli = ['2e6', '5e5', '1e7', '1e12']
Emoduli = ['2e6']

# Rigid beams elastic modulus (Pa: kg/m/s^2)
# rigidModuli = Emoduli[-1]
rigidModuli = Emoduli[-1]

# Fully developed inlet velocity profile (bool)
# LinearInletVel = True
LinearInletVel = True

# Velocity of the top wall (m/s)
# topVelocity = 3
topVelocity = 3

# Thickness of the hairs (m)
# beamThickness = 0.02
beamThickness = 0.02

mToNumCoeff=2

# In order to impose the linear velocity profile at the inlet, we need to creat a table which contains
# the inlet boundary points and the corresponding velocity at each point. numPoints is the number of points
# in that table
numPoints=100

# Width of the channel (m)
wChannel = 0.4

# Height of the channel (m)
hChannel = 1.0

# Length of the inlet region (m)
lInlet = 2

# Length of the outlet region (m)
lOutlet = 8


#***************************************************************
#*********************** Mesh parameters ***********************
#***************************************************************

beamYProgression = 0.25
xProgression = 0.25
inletProgression = 0.1
outletProgression = 0.06
beamXRefinementFactor = 2
                
#************************************************************
#*********************** Main Program ***********************
#************************************************************
initPath = dropbox_path
#initPath = os.getcwd()
for xSpacing in spacingX:
    
    spacingPath = os.path.join(initPath,str(xSpacing))
    os.mkdir(spacingPath)
    os.chdir(spacingPath)
    
    for hBeam in beamH:
        
        
        hBeamPath = os.path.join(spacingPath,str(hBeam))
        os.mkdir(hBeamPath)
        os.chdir(hBeamPath)
        

        for nBeams in beamsN:
            
            
            nBeamPath = os.path.join(hBeamPath,str(nBeams))
            os.mkdir(nBeamPath)
            os.chdir(nBeamPath)
            
            for moduli in Emoduli:
            
                moduliPath = os.path.join(nBeamPath,str(moduli))
                os.mkdir(moduliPath)
                os.chdir(moduliPath)
                
                os.system('cp -r %s %s'% (os.path.join(initPath,'case'),moduliPath))
                os.system('cp %s %s/rheologyProperties '% (os.path.join(initPath,str(moduli)),os.path.join(moduliPath,'case','solid','constant')))
                
                if moduli != rigidModuli:
                
                    PointsF = vertices2DFl(beamThickness, hBeam, wChannel, hChannel, lInlet, lOutlet, nBeams, xSpacing)
                    BlocksF = blocks2DFl(PointsF, xProgression, beamYProgression,inletProgression,outletProgression, beamXRefinementFactor,mToNumCoeff)
                    PointsS = vertices2DS(beamThickness, hBeam, wChannel, lInlet, lOutlet, nBeams, xSpacing)
                    BlocksS = blocks2DS(PointsS, beamYProgression, beamXRefinementFactor,mToNumCoeff)
                
                else:
                    
                    rigidBeamThickness = (nBeams-1)*xSpacing + nBeams*beamThickness
                    PointsF = vertices2DFl(rigidBeamThickness, hBeam, wChannel, hChannel, lInlet, lOutlet, 1, 0)
                    BlocksF = blocks2DFl(PointsF, xProgression, beamYProgression,inletProgression,outletProgression, beamXRefinementFactor,mToNumCoeff)
                    PointsS = vertices2DS(rigidBeamThickness, hBeam, wChannel, lInlet, lOutlet, 1, 0)
                    BlocksS = blocks2DS(PointsS, beamYProgression, beamXRefinementFactor,mToNumCoeff)
                    
                writeBlockMeshFl (PointsF, BlocksF)
                writeBlockMeshS (PointsS, BlocksS)
                os.system('mv %s/blockMeshDictF %s/blockMeshDict '% (moduliPath,os.path.join(moduliPath,'case','fluid','constant','polyMesh')))
                os.system('mv %s/blockMeshDictS %s/blockMeshDict '% (moduliPath,os.path.join(moduliPath,'case','solid','constant','polyMesh')))
                
                if LinearInletVel:
                    
                    writeUInletAndPoints(topVelocity,numPoints,hChannel,wChannel)
                    os.system('mv %s/U %s/U '% (moduliPath,os.path.join(moduliPath,'case','fluid','constant','boundaryData','inlet','0')))
                    os.system('mv %s/points %s/points '% (moduliPath,os.path.join(moduliPath,'case','fluid','constant','boundaryData','inlet')))
                    os.system('mv %s/U_linInlet %s/U '% (os.path.join(moduliPath,'case','fluid','0'),os.path.join(moduliPath,'case','fluid','0')))
                
                else:
                    
                    os.system('mv %s/U_zeroGrad %s/U '% (os.path.join(moduliPath,'case','fluid','0'),os.path.join(moduliPath,'case','fluid','0')))
                    
                #os.system('cp blockMeshDict* tmp/%d' % i)
                # Params info (writes to info.txt)
                info = "beamThickness = %s \nhBeam = %s\n\
                wChannel = %s \hChannel = %s \n\
                lInlet = %s \nlOutlet = %s \n\
                nBeams = %s \nxSpacing = %s \nbeamYProgression = %s \nxProgression = %s \nbeamRefinementCoeff = %s" % (beamThickness,hBeam,wChannel,hChannel,lInlet,lOutlet,nBeams,xSpacing,beamYProgression,xProgression,beamXRefinementFactor)

                # Writing calculation info into a files
                text_file = open("info.txt", "w")
                text_file.write(info)
                text_file.close()
                
                os.chdir(os.path.join(moduliPath,'case'))
                #os.system('source $HOME/foam/foam-extend-4.1/etc/bashrc')
                os.system('./makeSerialLinks fluid/ solid/')
                
                os.chdir(nBeamPath)
                
            os.chdir(hBeamPath)
            
        os.chdir(spacingPath)
        
    os.chdir(initPath)
  #   

  


FileNotFoundError: [Errno 2] No such file or directory: '/Users/Ali/Dropbox/paper/notebook/0.2/0.2/0.2/0.2/2/2e6/0.2/0.2/2/2e6/0.2/0.2/2/2e6/case'

In [2]:
# coding: utf-8
# Template script for FSI simulation of hairbeds 2D run
import os

#**************************************************************
#********************** Model parameters **********************
#**************************************************************

# Beam spacing (m) 
spacingX = [0.2,0.5,0.1]

# Beam height (m)
beamH = [0.2,0.35,0.5]

# Number of beams
beamsN = [2,4,6,8]

# Elastic modulus of the beams (Pa: kg/m/s^2)
Emoduli = ['2e6', '5e5', '1e7', '1e12']

# Rigid beams elastic modulus (Pa: kg/m/s^2)
rigidModuli = Emoduli[-1]

# Fully developed inlet velocity profile (bool)
LinearInletVel = True

# Velocity of the top wall (m/s)
topVelocity = 3

# Thickness of the hairs (m)
beamThickness = 0.02

# mToN

# In order to impose the linear velocity profile at the inlet, we need to creat a table which contains
# the inlet boundary points and the corresponding velocity at each point. numPoints is the number of points
# in that table
numPoints=100

# Width of the channel (m)
wChannel = 0.4

# Height of the channel (m)
hChannel = 1.0

# Length of the inlet region (m)
lInlet = 2

# Length of the outlet region (m)
lOutlet = 8


#***************************************************************
#*********************** Mesh parameters ***********************
#***************************************************************

beamYProgression = 0.25
xProgression = 0.25
inletProgression = 0.1
outletProgression = 0.06
beamXRefinementFactor = 2
                
#************************************************************
#*********************** Main Program ***********************
#************************************************************

initPath = os.getcwd()
for xSpacing in spacingX:
    
    spacingPath = os.path.join(initPath,str(xSpacing))
    os.mkdir(spacingPath)
    os.chdir(spacingPath)
    
    for hBeam in beamH:
        
        
        hBeamPath = os.path.join(spacingPath,str(hBeam))
        os.mkdir(hBeamPath)
        os.chdir(hBeamPath)
        

        for nBeams in beamsN:
            
            
            nBeamPath = os.path.join(hBeamPath,str(nBeams))
            os.mkdir(nBeamPath)
            os.chdir(nBeamPath)
            
            for moduli in Emoduli:
            
                moduliPath = os.path.join(nBeamPath,str(moduli))
                os.mkdir(moduliPath)
                os.chdir(moduliPath)
                
                os.system('cp -r %s %s'% (os.path.join(initPath,'case'),moduliPath))
                os.system('cp %s %s/rheologyProperties '% (os.path.join(initPath,str(moduli)),os.path.join(moduliPath,'case','solid','constant')))
                
                if moduli != rigidModuli:
                
                    PointsF = vertices2DFl(beamThickness, hBeam, wChannel, hChannel, lInlet, lOutlet, nBeams, xSpacing)
                    BlocksF = blocks2DFl(PointsF, xProgression, beamYProgression,inletProgression,outletProgression, beamXRefinementFactor,mToNumCoeff)
                    PointsS = vertices2DS(beamThickness, hBeam, wChannel, lInlet, lOutlet, nBeams, xSpacing)
                    BlocksS = blocks2DS(PointsS, beamYProgression, beamXRefinementFactor,mToNumCoeff)
                
                else:
                    
                    rigidBeamThickness = (nBeams-1)*xSpacing + nBeams*beamThickness
                    PointsF = vertices2DFl(rigidBeamThickness, hBeam, wChannel, hChannel, lInlet, lOutlet, 1, 0)
                    BlocksF = blocks2DFl(PointsF, xProgression, beamYProgression,inletProgression,outletProgression, beamXRefinementFactor,mToNumCoeff)
                    PointsS = vertices2DS(rigidBeamThickness, hBeam, wChannel, lInlet, lOutlet, 1, 0)
                    BlocksS = blocks2DS(PointsS, beamYProgression, beamXRefinementFactor,mToNumCoeff)
                    
                writeBlockMeshFl (PointsF, BlocksF)
                writeBlockMeshS (PointsS, BlocksS)
                os.system('mv %s/blockMeshDictF %s/blockMeshDict '% (moduliPath,os.path.join(moduliPath,'case','fluid','constant','polyMesh')))
                os.system('mv %s/blockMeshDictS %s/blockMeshDict '% (moduliPath,os.path.join(moduliPath,'case','solid','constant','polyMesh')))
                
                if LinearInletVel:
                    
                    writeUInletAndPoints(topVelocity,numPoints,hChannel,wChannel)
                    os.system('mv %s/U %s/U '% (moduliPath,os.path.join(moduliPath,'case','fluid','constant','boundaryData','inlet','0')))
                    os.system('mv %s/points %s/points '% (moduliPath,os.path.join(moduliPath,'case','fluid','constant','boundaryData','inlet')))
                    os.system('mv %s/U_linInlet %s/U '% (os.path.join(moduliPath,'case','fluid','0'),os.path.join(moduliPath,'case','fluid','0')))
                
                else:
                    
                    os.system('mv %s/U_zeroGrad %s/U '% (os.path.join(moduliPath,'case','fluid','0'),os.path.join(moduliPath,'case','fluid','0')))
                    
                #os.system('cp blockMeshDict* tmp/%d' % i)
                # Params info (writes to info.txt)
                info = "beamThickness = %s \nhBeam = %s\n\
                wChannel = %s \hChannel = %s \n\
                lInlet = %s \nlOutlet = %s \n\
                nBeams = %s \nxSpacing = %s \nbeamYProgression = %s \nxProgression = %s \nbeamRefinementCoeff = %s" % (beamThickness,hBeam,wChannel,hChannel,lInlet,lOutlet,nBeams,xSpacing,beamYProgression,xProgression,beamXRefinementFactor)

                # Writing calculation info into a files
                text_file = open("info.txt", "w")
                text_file.write(info)
                text_file.close()
                
                os.chdir(os.path.join(moduliPath,'case'))
                #os.system('source $HOME/foam/foam-extend-4.1/etc/bashrc')
                os.system('./makeSerialLinks fluid/ solid/')
                
                os.chdir(nBeamPath)
                
            os.chdir(hBeamPath)
            
        os.chdir(spacingPath)
        
    os.chdir(initPath)
            

 

In [16]:
# coding: utf-8
# Template script for FSI simulation of hairbeds 2D run
import os

#**************************************************************
#********************** Model parameters **********************
#**************************************************************

# Beam spacing (m) 
spacingX = [0.5]

# Beam height (m)
beamH = [0.5]

# Number of beams
beamsN = [2,8]

# Elastic modulus of the beams (Pa: kg/m/s^2)
Emoduli = ['5e5', '1e7']

# Rigid beams elastic modulus (Pa: kg/m/s^2)
rigidModuli = Emoduli[-1]

# Fully developed inlet velocity profile (bool)
LinearInletVel = True

# Velocity of the top wall (m/s)
topVelocity = 3

# Thickness of the hairs (m)
beamThickness = 0.02

# In order to impose the linear velocity profile at the inlet, we need to creat a table which contains
# the inlet boundary points and the corresponding velocity at each point. numPoints is the number of points
# in that table
numPoints=100

# Width of the channel (m)
wChannel = 0.4

# Height of the channel (m)
hChannel = 1.0

# Length of the inlet region (m)
lInlet = 2

# Length of the outlet region (m)
lOutlet = 8

mToNumCoeff=50
#***************************************************************
#*********************** Mesh parameters ***********************
#***************************************************************

beamYProgression = 0.25
xProgression = 0.25
inletProgression = 0.1
outletProgression = 0.06
beamXRefinementFactor = 2
                
#************************************************************
#*********************** Main Program ***********************
#************************************************************

initPath = os.getcwd()
for xSpacing in spacingX:
    
    spacingPath = os.path.join(initPath,str(xSpacing))
    os.mkdir(spacingPath)
    os.chdir(spacingPath)
    
    for hBeam in beamH:
        
        
        hBeamPath = os.path.join(spacingPath,str(hBeam))
        os.mkdir(hBeamPath)
        os.chdir(hBeamPath)
        

        for nBeams in beamsN:
            
            
            nBeamPath = os.path.join(hBeamPath,str(nBeams))
            os.mkdir(nBeamPath)
            os.chdir(nBeamPath)
            
            for moduli in Emoduli:
            
                moduliPath = os.path.join(nBeamPath,str(moduli))
                os.mkdir(moduliPath)
                os.chdir(moduliPath)
                
                os.system('cp -r %s %s'% (os.path.join(initPath,'case'),moduliPath))
                os.system('cp %s %s/rheologyProperties '% (os.path.join(initPath,str(moduli)),os.path.join(moduliPath,'case','solid','constant')))
                
                if moduli != rigidModuli:
                
                    PointsF = vertices2DFl(beamThickness, hBeam, wChannel, hChannel, lInlet, lOutlet, nBeams, xSpacing)
                    BlocksF = blocks2DFl(PointsF, xProgression, beamYProgression,inletProgression,outletProgression, beamXRefinementFactor,mToNumCoeff)
                    PointsS = vertices2DS(beamThickness, hBeam, wChannel, lInlet, lOutlet, nBeams, xSpacing)
                    BlocksS = blocks2DS(PointsS, beamYProgression, beamXRefinementFactor,mToNumCoeff)
                
                else:
                    
                    rigidBeamThickness = (nBeams-1)*xSpacing + nBeams*beamThickness
                    PointsF = vertices2DFl(rigidBeamThickness, hBeam, wChannel, hChannel, lInlet, lOutlet, 1, 0)
                    BlocksF = blocks2DFl(PointsF, xProgression, beamYProgression,inletProgression,outletProgression, beamXRefinementFactor,mToNumCoeff)
                    PointsS = vertices2DS(rigidBeamThickness, hBeam, wChannel, lInlet, lOutlet, 1, 0)
                    BlocksS = blocks2DS(PointsS, beamYProgression, beamXRefinementFactor,mToNumCoeff)
                    
                writeBlockMeshFl (PointsF, BlocksF)
                writeBlockMeshS (PointsS, BlocksS)
                os.system('mv %s/blockMeshDictF %s/blockMeshDict '% (moduliPath,os.path.join(moduliPath,'case','fluid','constant','polyMesh')))
                os.system('mv %s/blockMeshDictS %s/blockMeshDict '% (moduliPath,os.path.join(moduliPath,'case','solid','constant','polyMesh')))
                
                if LinearInletVel:
                    
                    writeUInletAndPoints(topVelocity,numPoints,hChannel,wChannel)
                    os.system('mv %s/U %s/U '% (moduliPath,os.path.join(moduliPath,'case','fluid','constant','boundaryData','inlet','0')))
                    os.system('mv %s/points %s/points '% (moduliPath,os.path.join(moduliPath,'case','fluid','constant','boundaryData','inlet')))
                    os.system('mv %s/U_linInlet %s/U '% (os.path.join(moduliPath,'case','fluid','0'),os.path.join(moduliPath,'case','fluid','0')))
                
                else:
                    
                    os.system('mv %s/U_zeroGrad %s/U '% (os.path.join(moduliPath,'case','fluid','0'),os.path.join(moduliPath,'case','fluid','0')))
                    
                #os.system('cp blockMeshDict* tmp/%d' % i)
                # Params info (writes to info.txt)
                info = "beamThickness = %s \nhBeam = %s\n\
                wChannel = %s \hChannel = %s \n\
                lInlet = %s \nlOutlet = %s \n\
                nBeams = %s \nxSpacing = %s \nbeamYProgression = %s \nxProgression = %s \nbeamRefinementCoeff = %s" % (beamThickness,hBeam,wChannel,hChannel,lInlet,lOutlet,nBeams,xSpacing,beamYProgression,xProgression,beamXRefinementFactor)

                # Writing calculation info into a files
                text_file = open("info.txt", "w")
                text_file.write(info)
                text_file.close()
                
                os.chdir(os.path.join(moduliPath,'case'))
                #os.system('source $HOME/foam/foam-extend-4.1/etc/bashrc')
                os.system('./makeSerialLinks fluid/ solid/')
                
                os.chdir(nBeamPath)
                
            os.chdir(hBeamPath)
            
        os.chdir(spacingPath)
        
    os.chdir(initPath)
            

 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/lib/python2.7/inspect.py", line 1009, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 483, in getmodule
    file = getabsfile(object, _fi

IndexError: string index out of range

In [17]:
os.getcwd()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/lib/python2.7/inspect.py", line 1009, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 483, in getmodule
    file = getabsfile(object, _fi

IndexError: string index out of range

In [66]:
rigidModuli == '1e12'

True

In [70]:
pwd

u'/home/ali1/Dropbox/OF/hairBedSims/2DSims'

In [14]:
cd 0.4

[Errno 2] No such file or directory: '0.4'
/home/ali1/Dropbox/OF/hairBedSims/2DSims


In [7]:
import os
beamThickness = 0.03
hBeam = 0.45
wChannel = 0.4
hChannel = 0.5
lInlet = 0.5
lOutlet = 0.5
nBeams = 14
xSpacing = 0.4
beamYProgression = 0.2
xProgression = 0.2
beamXRefinementFactor = 2
PointsF =  vertices2DFl(beamThickness, hBeam, wChannel, hChannel, lInlet, lOutlet, nBeams, xSpacing)
BlocksF = blocks2DFl(PointsF, xProgression, beamYProgression, beamXRefinementFactor)
PointsS = vertices2DS(beamThickness, hBeam, wChannel, lInlet, lOutlet, nBeams, xSpacing)
BlocksS = blocks2DS(PointsS, beamYProgression, beamXRefinementFactor)
writeBlockMeshFl (PointsF, BlocksF)
writeBlockMeshS (PointsS, BlocksS)
"""os.system('cp blockMeshDict* tmp/%d' % i)"""
# Params info (writes to info.txt)
info = "beamThickness = %s \nhBeam = %s\n\
wChannel = %s \hChannel = %s \n\
lInlet = %s \nlOutlet = %s \n\
nBeams = %s \nxSpacing = %s \nbeamYProgression = %s \nxProgression = %s \nbeamRefinementCoeff = %s" % (beamThickness,hBeam,wChannel,hChannel,lInlet,lOutlet,nBeams,xSpacing,beamYProgression,xProgression,beamXRefinementFactor)

# Writing calculation info into a files
text_file = open("info.txt", "w")
text_file.write(info)
text_file.close()

In [58]:
Emoduli = [1e5, 3e5, 1e6]
spacingPath = os.getcwd()
for hBeam in beamH:
        
    hBeamPath = os.path.join(spacingPath,str(hBeam))
    for nBeams in beamsN:
            
        nBeamPath = os.path.join(hBeamPath,str(nBeams))
        for moduli in Emoduli:
            
            moduliPath = os.path.join(nBeamPath,str(moduli))
            os.system('./makeSerialLinks fluid/ solid/')
            os.chdir(nBeamPath)
            
        os.chdir(hBeamPath)
        
    os.chdir(spacingPath)
        

            

In [19]:
import os
os.getcwd()

'/home/ali1/Dropbox/OF/hairBedSims/2DSims'

In [62]:
pwd

u'/home/ali1/Dropbox/OF/hairBedSims/2DSims'

In [24]:
wChannel

0.4

In [31]:
v = 5
numPoints = 100
h = hChannel
U_boundaryData = np.linspace(0,v,numPoints+1)
points_boundaryData = np.linspace(0,h,numPoints+1)


In [32]:
U_boundaryData

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.05,
       1.1 , 1.15, 1.2 , 1.25, 1.3 , 1.35, 1.4 , 1.45, 1.5 , 1.55, 1.6 ,
       1.65, 1.7 , 1.75, 1.8 , 1.85, 1.9 , 1.95, 2.  , 2.05, 2.1 , 2.15,
       2.2 , 2.25, 2.3 , 2.35, 2.4 , 2.45, 2.5 , 2.55, 2.6 , 2.65, 2.7 ,
       2.75, 2.8 , 2.85, 2.9 , 2.95, 3.  , 3.05, 3.1 , 3.15, 3.2 , 3.25,
       3.3 , 3.35, 3.4 , 3.45, 3.5 , 3.55, 3.6 , 3.65, 3.7 , 3.75, 3.8 ,
       3.85, 3.9 , 3.95, 4.  , 4.05, 4.1 , 4.15, 4.2 , 4.25, 4.3 , 4.35,
       4.4 , 4.45, 4.5 , 4.55, 4.6 , 4.65, 4.7 , 4.75, 4.8 , 4.85, 4.9 ,
       4.95, 5.  ])

In [33]:
U_new = np.zeros((numPoints,3))

In [34]:
U_new[:,0] = U_boundaryData[1:]

In [35]:
U_new

array([[0.05, 0.  , 0.  ],
       [0.1 , 0.  , 0.  ],
       [0.15, 0.  , 0.  ],
       [0.2 , 0.  , 0.  ],
       [0.25, 0.  , 0.  ],
       [0.3 , 0.  , 0.  ],
       [0.35, 0.  , 0.  ],
       [0.4 , 0.  , 0.  ],
       [0.45, 0.  , 0.  ],
       [0.5 , 0.  , 0.  ],
       [0.55, 0.  , 0.  ],
       [0.6 , 0.  , 0.  ],
       [0.65, 0.  , 0.  ],
       [0.7 , 0.  , 0.  ],
       [0.75, 0.  , 0.  ],
       [0.8 , 0.  , 0.  ],
       [0.85, 0.  , 0.  ],
       [0.9 , 0.  , 0.  ],
       [0.95, 0.  , 0.  ],
       [1.  , 0.  , 0.  ],
       [1.05, 0.  , 0.  ],
       [1.1 , 0.  , 0.  ],
       [1.15, 0.  , 0.  ],
       [1.2 , 0.  , 0.  ],
       [1.25, 0.  , 0.  ],
       [1.3 , 0.  , 0.  ],
       [1.35, 0.  , 0.  ],
       [1.4 , 0.  , 0.  ],
       [1.45, 0.  , 0.  ],
       [1.5 , 0.  , 0.  ],
       [1.55, 0.  , 0.  ],
       [1.6 , 0.  , 0.  ],
       [1.65, 0.  , 0.  ],
       [1.7 , 0.  , 0.  ],
       [1.75, 0.  , 0.  ],
       [1.8 , 0.  , 0.  ],
       [1.85, 0.  , 0.  ],
 

In [58]:
cd /home/ali1/Dropbox/OF/hairBedSims/2DSims

/home/ali1/Dropbox/OF/hairBedSims/2DSims


In [59]:
topVelocity = 5
numPoints = 100
hChannel = 1
wChannel = 0.4

def writeUInletAndPoints(topVelocity,numPoints,hChannel,wChannel):

    U = np.zeros((numPoints,3))
    points = np.zeros((numPoints,3))
    U[:,0] = np.linspace(0,topVelocity,numPoints+1)[1:]
    points[:,1] = np.linspace(0,hChannel,numPoints+1)[1:]

    info_before_U = "FoamFile\n\
{\n\
    version     2.0;\n\
    format      ascii;\n\
    class       vectorAverageField;\n\
    object      values;\n\
}\n\
\n\
// Average\n\
(0 0 0)\n\
\n\
// Data on points\n\
%d \n\
(\n" % (numPoints*2)

    info_before_points = "FoamFile\n\
{\n\
    version     2.0;\n\
    format      ascii;\n\
    class       vectorField;\n\
    object      points;\n\
}\n\
\n\
(\n"  

    f_U = open('U',"w")
    f_U.write(info_before_U)
    for i in range(numPoints):
        f_U.write(('(%4.2f %4.2f %4.2f)\n') % (U[i][0],U[i][1],U[i][2] ))
    f_U.write('\n')
    for i in range(numPoints):
        f_U.write(('(%4.2f %4.2f %4.2f)\n') % (U[i][0],U[i][1],U[i][2] ))
    f_U.write('\n)\n')
    f_U.close()

    f_points = open('points',"w")
    f_points.write(info_before_points)
    for i in range(numPoints):
        f_points.write(('(%4.2f %4.2f %4.2f)\n') % (points[i][0],points[i][1],-wChannel))
    f_points.write('\n')
    for i in range(numPoints):
        f_points.write(('(%4.2f %4.2f %4.2f)\n') % (points[i][0],points[i][1],points[i][2] ))
    f_points.write('\n)\n')
    f_points.close()

    pass

In [60]:
 writeUInletAndPoints(topVelocity,numPoints,hChannel,wChannel)

In [51]:
pwd

u'/home/ali1/Dropbox/OF/hairBedSims/2DSims'